<a href="https://colab.research.google.com/github/krishnatejak2/EVA4/blob/master/EVA4_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_1_4_3 = nn.Conv2d(1, 4, 3, padding=1) #,  stride = 2)     
        self.conv_4_8_3 = nn.Conv2d(4, 8, 3, padding=1)#, stride = 2)
        self.conv_8_10_3 = nn.Conv2d(8, 10, 3, padding=1) #,  stride = 2)           
        self.conv_10_12_3 = nn.Conv2d(10, 12, 3, padding=1) #,stride=2)
        self.conv_16_32_3 = nn.Conv2d(16, 32, 3, padding=1) #,stride=2)

        self.conv_12_16_3_nopad = nn.Conv2d(12, 16, 3)#, padding=1) #,stride=2)
        self.conv_16_20_3_nopad = nn.Conv2d(16, 20, 3)#, padding=1) #,stride=2)
        self.conv_20_10_3 = nn.Conv2d(20, 10, 3)#, padding=1) #,stride=2)
        
        self.pool = nn.MaxPool2d(2, 2)
                        
        self.drop = nn.Dropout2d(0.1)
        self.batch_nn_4 = nn.BatchNorm2d(4)
        self.batch_nn_8 = nn.BatchNorm2d(8)
        self.batch_nn_16 = nn.BatchNorm2d(16)
        self.batch_nn_10 = nn.BatchNorm2d(10)
        self.batch_nn_12 = nn.BatchNorm2d(12)
        self.batch_nn_20 = nn.BatchNorm2d(20)
        self.batch_nn_32 = nn.BatchNorm2d(32)
        self.batch_nn_64 = nn.BatchNorm2d(64)
        # self.GAP_10 = nn.AdaptiveAvgPool3d((10,1,1))

    def forward(self, x):
      x = self.drop(self.batch_nn_4(F.relu(self.conv_1_4_3(x))))        # Output : 28x28x4
      x = self.drop(self.batch_nn_8(F.relu(self.conv_4_8_3(x))))        # Output : 28x28x8
      x = self.pool(x)                                                  # Output : 14x14x8          
      x = self.drop(self.batch_nn_10(F.relu(self.conv_8_10_3(x))))      # Output : 14x14x10           
      x = self.drop(self.batch_nn_12(F.relu(self.conv_10_12_3(x))))     # Output : 14x14x12
      x = self.pool(x)                                                  # Output : 7x7x12
      x = self.drop(self.batch_nn_16(F.relu(self.conv_12_16_3_nopad(x))))     # Output : 5x5x16   
      x = self.drop(self.batch_nn_20(F.relu(self.conv_16_20_3_nopad(x))))     # Output : 3x3x20
      # x = self.batch_nn_32(self.drop(F.relu(self.conv_10_10_4(x))))   # Output : 1x1x10
      x = self.conv_20_10_3(x)                                          # Output : 1x1x10
      # x = self.GAP_10(x)                                      
      x = x.view(-1,10)
      return F.log_softmax(x)


        # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # x = self.conv7(x)
        # x = x.view(-1, 10)
        # return F.log_softmax(x)

In [3]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
       BatchNorm2d-2            [-1, 4, 28, 28]               8
         Dropout2d-3            [-1, 4, 28, 28]               0
            Conv2d-4            [-1, 8, 28, 28]             296
       BatchNorm2d-5            [-1, 8, 28, 28]              16
         Dropout2d-6            [-1, 8, 28, 28]               0
         MaxPool2d-7            [-1, 8, 14, 14]               0
            Conv2d-8           [-1, 10, 14, 14]             730
       BatchNorm2d-9           [-1, 10, 14, 14]              20
        Dropout2d-10           [-1, 10, 14, 14]               0
           Conv2d-11           [-1, 12, 14, 14]           1,092
      BatchNorm2d-12           [-1, 12, 14, 14]              24
        Dropout2d-13           [-1, 12, 14, 14]               0
        MaxPool2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, 
    shuffle=True, 
    **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, 
    shuffle=True, 
    **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):#,scheduler):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data*(1-0.1))                                      # multiplying the input data with the correction for drop out
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9,  nesterov= True)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=1,gamma = 0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(1, 20):
    # scheduler.step()
    train(model, device, train_loader, optimizer, epoch)#,scheduler)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0658, Accuracy: 9782/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0455, Accuracy: 9853/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0227, Accuracy: 9917/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0215, Accuracy: 9927/10000 (99%)



loss=0.0038731396198272705 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.48it/s]



Test set: Average loss: 0.0221, Accuracy: 9910/10000 (99%)



AttributeError: ignored